# Transformation from thermal circuit to state-space: cmparison betwen dm4bem and pd_dm4bem

Example from `03CubicBuilding.ipynb` and `041tc2ss_ass.ipynb`.

Treated with:
    `dm4bem.tc2ss`: without assembling
    `pd_dm4bem.tc2ss`: with assembling

Use the same notations figure TC.svg

In [1]:
import numpy as np
import pandas as pd
import dm4bem
import pd_dm4bem

## Thermal circuit for `dm4bem.tc2ss()`

Let's consider the thermal circuit TC = {A, G, C, b, f, y} used to obtain the state-space representation `dm4bem.tc2ss()` (Figure 1). 

![tc_sym](./bldg_tc2ss_pd/TC.svg)
> Figure 1. Thermal circuit for `dm4bem.tc2ss()`.

In [2]:
"""
dm4bem.tc2ss: without assembling
"""
TC0 = pd.read_csv("./bldg_tc2ss_pd/TC0.csv")
TC1 = pd.read_csv("./bldg_tc2ss_pd/TC1.csv")
TC2 = pd.read_csv("./bldg_tc2ss_pd/TC2.csv")
TC3 = pd.read_csv("./bldg_tc2ss_pd/TC3.csv")

wS = pd.read_csv("./bldg/walls_out.csv")

w = pd.read_csv("./bldg/wall_types.csv")
w0 = w[w['type'] == 0]
w0.set_index('Material', inplace=True)

A = np.zeros([12, 8])       # n° of branches X n° of nodes
A[0, 7], A[0, 1] = -1, 1
A[1, 0] = 1
A[2, 0], A[2, 1] = -1, 1
A[3, 1], A[3, 2] = -1, 1
A[4, 2] = 1
A[5, 2] = 1
A[6, 3] = 1
A[7, 3], A[7, 4] = -1, 1
A[8, 4], A[8, 5] = -1, 1
A[9, 5], A[9, 6] = -1, 1
A[10, 6], A[10, 7] = -1, 1
A[11, 7], A[11, 2] = -1, 1


G = np.zeros(A.shape[0])
G[0] = TC0.loc[0]['G']
G[1] = TC1.loc[0]['G']
G[2] = TC1.loc[1]['G']
G[3] = TC1.loc[2]['G']
G[4] = TC2.loc[0]['G']
G[5] = TC3.loc[0]['G']
G[6] = wS.loc[0]['h0'] * wS.loc[0]['Area']
G[7] = G[8] = w0.loc[
    'Insulation']['Conductivity'] / (w0.loc[
        'Insulation']['Width'] / 2) * wS.loc[0]['Area']
G[9] = G[10] = w0.loc[
    'Concrete']['Conductivity'] / (w0.loc[
        'Concrete']['Width'] / 2) * wS.loc[0]['Area']
G[11] = wS.loc[0]['h1'] * wS.loc[0]['Area']


C = np.zeros(A.shape[1])
C[0] = TC1.loc[TC1['A'] == 'C', 'θg']
C[2] = TC2.loc[TC2['A'] == 'C', 'θ0']
C[4] = w0.loc[
    'Insulation']['Density'] * w0.loc[
        'Insulation']['Specific heat'] * w0.loc[
            'Insulation']['Width'] * wS.loc[0]['Area']
C[6] = w0.loc[
    'Concrete']['Density'] * w0.loc[
        'Concrete']['Specific heat'] * w0.loc[
            'Concrete']['Width'] * wS.loc[0]['Area']

b = np.zeros(12)        # all branches
b[[1, 4, 5, 6]] = 1     # branches with temperature sources

f = np.zeros(8)         # all nodes
f[[0, 2, 3, 7]] = 1     # nodes with heat-flow sources

y = np.zeros(8)         # nodes
y[[2, 7]] = 1           # nodes: in wall surface, in air

[As, Bs, Cs, Ds] = dm4bem.tc2ss(A, np.diag(G), np.diag(C), b, f, y)
print('As = \n', As, '\n')
print('Bs = \n', Bs, '\n')
print('Cs = \n', Cs, '\n')
print('Ds = \n', Ds, '\n')

As = 
 [[-2.41440668e-04  6.80230103e-05  0.00000000e+00  2.11779883e-05]
 [ 2.28632896e-03 -9.42774014e-03  0.00000000e+00  6.86363341e-03]
 [ 0.00000000e+00  0.00000000e+00 -2.44403914e-04  1.20952723e-04]
 [ 1.26607539e-06  1.22080436e-05  1.59079125e-06 -1.50649103e-05]] 

Bs = 
 [[1.52239669e-04 0.00000000e+00 0.00000000e+00 0.00000000e+00
  9.18273646e-07 0.00000000e+00 0.00000000e+00 3.36158545e-08]
 [0.00000000e+00 2.77777778e-04 0.00000000e+00 0.00000000e+00
  0.00000000e+00 3.08641975e-05 0.00000000e+00 1.08946562e-05]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 1.23451191e-04
  0.00000000e+00 0.00000000e+00 1.09734392e-07 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 3.35093036e-08]] 

Cs = 
 [[0.         1.         0.         0.        ]
 [0.03660767 0.35298686 0.         0.61040547]] 

Ds = 
 [[0.        0.        0.        0.        0.        0.        0.
  0.       ]
 [0.        0.        0

## Thermal circuit for `pd_dm4bem.tc2ss()`

Let's consider the thermal circuit TCl obtained by assembling according to  used to obtain the state-space representation `dm4bem.tc2ss()` (Figure 1). 

![tc_sym](./bldg_tc2ss_pd/TC.svg)
> Figure 1. Thermal circuit for `dm4bem.tc2ss()`.

In [3]:
# Dissasembled thermal circuits
folder_path = "bldg_tc2ss_pd"
TCd = pd_dm4bem.bldg2TCd(folder_path,
                         TC_auto_number=True)
# For non auto-numbering of thermal circuits TC
# TCd = pd_dm4bem.bldg2TCd(folder_path, TC_auto_number=False)

# Assembled thermal circuit
# from assembly_matrix.csv')
ass_mat = pd.read_csv(folder_path + '/assembly_matrix.csv')
TCm = pd_dm4bem.assemble_TCd_matrix(TCd, ass_mat)

# from assembly_lists.csv'
ass_lists = pd.read_csv(folder_path + '/assembly_lists.csv')
ass_mat = pd_dm4bem.assemble_lists2matrix(ass_lists)
TCl = pd_dm4bem.assemble_TCd_matrix(TCd, ass_mat)

# State-space from TC
[Al, Bl, Cl, Dl, ul] = pd_dm4bem.tc2ss(TCl)

In [4]:
print('absolute maximum difference between As and Al:')
(As - Al).abs().max().max()

absolute maximum difference between As and Al:


0.0

In [5]:
print('absolute maximum difference between Bs and Bl:')
(Bs - Bl).abs().max().max()

absolute maximum difference between Bs and Bl:


0.0

In [6]:
print('absolute maximum difference between Cs and Cl:')
(Cs - Cl).abs().max().max()

absolute maximum difference between Cs and Cl:


0.0

In [7]:
print('absolute maximum difference between Ds and Dl:')
(Ds - Dl).abs().max().max()

absolute maximum difference between Ds and Dl:


0.0